In [25]:
import config
import pymongo
import numpy as np
import pandas as pd
import json
import re
import simplejson
from math import isnan
import ast
import attributes


from pandas_profiling import ProfileReport
from ipywidgets import interact, Dropdown

/home/jovyan/attributes.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_tmp['Object'] = obj


In [26]:
myclient = pymongo.MongoClient(str(config.MONGO_URI))
stagingDb = myclient[str(config.DB_STAGING)]
analyseDb = myclient[str(config.DB_ANALYSE)]
stagingCol = stagingDb[config.COLL_PLAATJES]
stagingOud = stagingDb[config.COLL_STAGING_OUD]
stagingNieuw = stagingDb[config.COLL_STAGING_NIEUW]
analyseCol = analyseDb[config.COLL_ANALYSE]
analyseColClean = analyseDb[config.COLL_ANALYSE_CLEAN]
metaCollection = stagingDb['Kolominformatie']

AIRFLOW_WASSTRAAT_CONFIG = "./wasstraat_config/Wasstraat_Config_Harmonize.xlsx"
AIRFLOW_WASSTRAAT_CONFIG_NEW = "./wasstraat_config/Wasstraat_Config_Harmonize_new.xlsx"

SUGGESTIE_XL = "./wasstraat_config/Wasstraat_Attribuut_Suggestie_v2.xlsx"

In [27]:
df_attributes = attributes.getAllAttributes()
df_attributes

,Object,Kolommen,omschrijvingen,count,projecten,Attribute,teller,percentage_gevuld
0,Aardewerk,(none),"[KEUZELIJST AARDEWERK 2 VOORWERP, KEUZELIJST A...",25,"[DC21, DC33, DC72, DC05, DC37, DC24_STADSKANTO...",,NaN,NaN
1,Aardewerk,10a,[doosnummer],16,"[DC21, DC33, DC72, DC05, DC37, DC97, HHD2, DC2...",doosnr,6222.0,65.0
2,Aardewerk,10b,[tekeningnummer],16,"[DC21, DC33, DC72, DC05, DC37, DC27, HHD2, DC2...",tekeningnr,617.0,6.0
3,Aardewerk,10c,[dianummer],16,"[DC21, DC33, DC72, DC05, DC37, DC27, HHD2, DC2...",fotonr,327.0,3.0
4,Aardewerk,10d,[digifotonummer],16,"[DC21, DC33, DC72, DC05, DC37, DC97, HHD2, DC2...",fotonr,329.0,3.0
...,...,...,...,...,...,...,...,...
1435,Vulling,VKL,[Verbrande Klei],6,"[DC24_STADSKANTOOR, DC154, DC179, DC008]",,NaN,NaN
1436,Vulling,VOEGMAT,[voegmateriaal],6,"[DC24_STADSKANTOOR, DC008, DC179]",,NaN,NaN
1437,Vulling,VONDSTNUMMERS,[vondstnummers uit deze vulling],7,"[HHD2, DC30, DC29, DC97, DC93, DC112, DB34]",,NaN,NaN
1438,Vulling,VULLING,[vullingnummer],6,"[DC24_STADSKANTOOR, DC154, DC179, DC008]",vullingnr,NaN,NaN


In [28]:
xl_config = pd.read_excel(AIRFLOW_WASSTRAAT_CONFIG, None);
df_table = xl_config['Artefact']
df_table['Kolommen'] = df_table.apply(lambda x: ast.literal_eval(x['Kolommen']), axis=1)

df_table.head(5)

,Unnamed: 0,Attribute,Kolommen,Opmerkingen
0,0.0,ABRcodering,[ABR],Zelf toevoeging
1,1.0,aantal,"[TOTAANTAL, AANTAL, 4b1, FRAGM]",NaN
2,2.0,afmetingen,"[AFMETING, FORMAAT IN MM VERTIKAAL, FORMAAT IN...",NaN
3,3.0,artefactnr,"[ARTEFACT, SUBNO, ID]",In RAAP heb je zowel ARTEFACT en ook SUBNO. De...
4,4.0,beschrijving,"[4bvoorwerp beschrijving, 4b, BESCHR, VOORZIJD...",NaN


In [29]:
xl_sugg = pd.read_excel(SUGGESTIE_XL, None);
df_sugg = xl_sugg['Aardewerk']
df_sugg.head(5)

,Unnamed: 0,Object,Kolommen,omschrijvingen,count,projecten,Attribute,teller,percentage_gevuld,sugg_kolom,sugg_omschr,new_attr,new_attr_artefact
0,104,Aardewerk,VORM,['vorm'],4,"['DC179', 'DC008', 'DB34', 'DC24_STADSKANTOOR']",NaN,148.0,2.0,vorm,vorm,vorm,NaN
1,103,Aardewerk,VOORWERP,['type voorwerp'],1,['DB34'],NaN,NaN,NaN,voorwerp,type,NaN,typevoorwerp
2,129,Aardewerk,vondstnr,[],1,['DC24_STADSKANTOOR'],vondstnr,880.0,11.0,vondstnr,NaN,NaN,NaN
3,102,Aardewerk,VONDSTNO,['vondstnummer'],11,"['DC72', 'DC112', 'DC36', 'DC20', 'DB34', 'DC0...",vondstnr,7090.0,88.0,vondstno,vondstnr,NaN,NaN
4,128,Aardewerk,vondst,[],1,['DC24_STADSKANTOOR'],NaN,880.0,11.0,vondst,NaN,NaN,vondstnr


In [36]:
def combineDefs(df_config, df_sugg):
    df_config = df_config.append(df_sugg)
    df_config = df_config.groupby('Attribute').agg({'Kolommen':lambda x: list(x), 'Opmerkingen':lambda x: list(x)}).reset_index()
    df_config['Kolommen'] = df_config.apply(lambda x: list(set([item for sublist in x['Kolommen'] for item in sublist])), axis=1)
    df_config['Opmerkingen'] = df_config.apply(lambda x: ','.join([elem for elem in x['Opmerkingen'] if not pd.isnull(elem)]), axis=1)
    return df_config

    
kolommen = ['index','Attribute', 'Kolommen', 'Opmerkingen']
sheets_sugg = xl_sugg.keys()
sheets_config = xl_config.keys()
new_config = {}


df_artefact = pd.DataFrame()
for sheet in sheets_config:
    df_config = pd.read_excel(AIRFLOW_WASSTRAAT_CONFIG, sheet)
    if sheet in sheets_sugg:
        print(sheet)
        df_sugg = pd.read_excel(SUGGESTIE_XL, sheet)
        df_sugg = df_sugg.groupby('new_attr_artefact').agg({'Kolommen':lambda x: list(set(x))})
        df_artefact = df_artefact.append(df_sugg.copy()) 
        
        # Now find suggestions for this one tab
        df_sugg = pd.read_excel(SUGGESTIE_XL, sheet)
        df_sugg = df_sugg.groupby('new_attr').agg({'Kolommen':lambda x: list(set(x))})
        df_sugg.reset_index(inplace=True)
        df_sugg['Opmerkingen'] = None
        df_sugg['index'] = None
        df_sugg.columns = kolommen
        
        #Check Config
        if len(df_config) > 0:
            df_config['Kolommen'] = df_config.apply(lambda x: ast.literal_eval(x['Kolommen']), axis=1)
            if len(df_config.columns) < 4:
                df_config['Opmerkingen'] = None
            df_config.columns = kolommen
        else: 
            df_config = pd.DataFrame(columns = kolommen)

        df_config = combineDefs(df_config, df_sugg)
    
    new_config[sheet] = df_config
    
#df_sugg = pd.read_excel(SUGGESTIE_XL, 'Aardewerk')
#df_sugg = df_sugg.groupby('new_attr_artefact').agg({'Kolommen':lambda x: list(set(x))})
#df_sugg

df_config_artefact = pd.read_excel(AIRFLOW_WASSTRAAT_CONFIG, 'Artefact')
df_config_artefact['Kolommen'] = df_config_artefact.apply(lambda x: ast.literal_eval(x['Kolommen']), axis=1)
df_config_artefact.columns = kolommen

df_artefact.reset_index(inplace=True)
df_artefact['Opmerkingen'] = None
df_artefact.columns = kolommen

df_config_artefact = combineDefs(df_config_artefact, df_artefact)
new_config['Artefact'] = df_config_artefact


new_config['Glas']

Hoorn
Menselijk_Materiaal


TypeError: unhashable type: 'list'

In [33]:
with pd.ExcelWriter(AIRFLOW_WASSTRAAT_CONFIG_NEW) as writer:  
    for sheet in new_config.keys():
        new_config[sheet].to_excel(writer, sheet_name=sheet)


In [22]:
txt = '''artefactnr = Column(Integer)
    beschrijving = Column(Text)
    opmerkingen = Column(Text)
    typevoorwerp = Column(String(200))
    typecd = Column(String(200))
    functievoorwerp = Column(String(200))
    origine = Column(String(200))
    dateringvanaf = Column(Integer)
    dateringtot = Column(Integer)
    datering = Column(String(200))
    conserveren = Column(Integer)
    exposabel = Column(Integer)
    literatuur = Column(String(200))
    putnr = Column(Integer)
    subnr = Column(Integer) 
    restauratie = Column(Integer)'''

lst_aanw = [line.rpartition('=')[0].strip() for line in iter(txt.splitlines())] 
lst_aanw

['artefactnr',
 'beschrijving',
 'opmerkingen',
 'typevoorwerp',
 'typecd',
 'functievoorwerp',
 'origine',
 'dateringvanaf',
 'dateringtot',
 'datering',
 'conserveren',
 'exposabel',
 'literatuur',
 'putnr',
 'subnr',
 'restauratie']

In [24]:
col = 'typevoorwerp = Column(String(200))'

df_artf = new_config['Aardewerk']
for index, row in df_artf.iterrows():
    if not row['Attribute'] in lst_aanw:
        print(row['Attribute'] + ' = Column(String(200))')
    


aantal = Column(String(200))
baksel = Column(String(200))
bakseltype = Column(String(200))
bodem = Column(String(200))
categorie = Column(String(200))
codes = Column(String(200))
compleetheid = Column(String(200))
decoratie = Column(String(200))
diameter = Column(String(200))
diversen = Column(String(200))
fragment = Column(String(200))
glazuur = Column(String(200))
grootste_diameter = Column(String(200))
hoogte = Column(String(200))
kleur = Column(String(200))
mai = Column(String(200))
materiaal = Column(String(200))
materiaalsoort = Column(String(200))
max_diameter = Column(String(200))
oor = Column(String(200))
past_aan = Column(String(200))
percentage = Column(String(200))
productiewijze = Column(String(200))
rand = Column(String(200))
rand_bodem = Column(String(200))
rand_diameter = Column(String(200))
versiering = Column(String(200))
vorm = Column(String(200))


In [19]:
for line in iter(txt.splitlines()):
    stripped_line = line.strip()
    print(stripped_line.rpartition('=')[0].strip())


artefactnr
beschrijving
opmerkingen
typevoorwerp
typecd
functievoorwerp
origine
dateringvanaf
dateringtot
datering
conserveren
exposabel
literatuur
putnr
subnr
restauratie



In [ ]:
ABR-codering = Column(String(200))
aantal = Column(Integer)
afmetingen = Column(String(200))
bibliografie = Column(String(200))
catalogus = Column(String(200))
compleetheid = Column(String(200))
conservering = Column(String(200))
diversen = Column(String(200))
gewicht = Column(String(200))
groep = Column(String(200))
mai = Column(String(200))
maten = Column(String(200))
materiaal = Column(String(200))
naam_voorwerp = Column(String(200))
plek = Column(String(200))
publicatiecode = Column(String(200))
soortvoorwerp = Column(String(200))
typenaam = Column(String(200))
versiering = Column(String(200))
vondstomstandigheden = Column(String(200))
weggegooid = Column(Integer)